# Dividend Yield (DY): 5 anos e 12 meses

Esse código lê dois arquivos CSV — um com os preços atuais das ações (precos_acoes.csv) e outro com o total de dividendos pagos nos últimos 5 anos e nos últimos 12 meses (dividendos_ano_resumo.csv) —, padroniza os tickers para que coincidam mesmo quando um arquivo usa sufixo .SA e o outro não, junta as informações, calcula o Dividend Yield médio dos últimos 5 anos (DY5anos) e o Dividend Yield dos últimos 12 meses (DY12M) como porcentagem sobre o preço atual, arredonda os valores, exibe os 5 primeiros resultados no console e salva um arquivo dividend_yield.csv contendo apenas as colunas ticker, DY5anos e DY12M na pasta ../data/.

In [15]:
import pandas as pd
import os

# Carregar os arquivos CSV
precos = pd.read_csv("../data/precos_acoes.csv")
div = pd.read_csv("../data/dividendos_ano_resumo.csv")

# Normalizar tickers para merge
precos["ticker_base"] = (
    precos["ticker"].astype(str).str.upper().str.strip().str.replace(".SA", "", regex=False)
)
div["ticker_base"] = div["ticker"].astype(str).str.upper().str.strip()

# Garantir tipos numéricos
precos["fechamento_atual"] = pd.to_numeric(precos["fechamento_atual"], errors="coerce")
div["valor_5anos"] = pd.to_numeric(div["valor_5anos"], errors="coerce")
div["valor_12m"] = pd.to_numeric(div["valor_12m"], errors="coerce")

# Merge para ter todos os tickers do arquivo de preços
df = precos.merge(div[["ticker_base", "valor_5anos", "valor_12m"]], on="ticker_base", how="left")

# Calcular DYs
df["DY5anos"] = (((df["valor_5anos"] / 5) / df["fechamento_atual"]) * 100).where(df["fechamento_atual"] > 0)
df["DY12M"]   = ((df["valor_12m"] / df["fechamento_atual"]) * 100).where(df["fechamento_atual"] > 0)

# Arredondar
df["DY5anos"] = df["DY5anos"].round(2)
df["DY12M"] = df["DY12M"].round(2)

# Selecionar apenas as colunas desejadas
df_final = df[["ticker", "DY5anos", "DY12M"]]

# Mostrar primeiros 5
print(df_final.head(5))

# Salvar no caminho solicitado
os.makedirs("../data", exist_ok=True)
df_final.to_csv("../data/dividend_yield.csv", index=False)
print("Arquivo salvo em: ../data/dividend_yield.csv")


     ticker  DY5anos   DY12M
0  VIVT3.SA     3.71    3.11
1  TIMS3.SA   305.90  379.30
2  FIQE3.SA     3.23    2.53
3  BRST3.SA     1.45    7.27
4  TELB4.SA      NaN     NaN
Arquivo salvo em: ../data/dividend_yield.csv
